# Proceso de Conversion de los Datasets originales a los archivos CSV que serviran de datos para el juego

Los datasets elegidos para el desarrollo del juego son: 
* GRUPO A:
    - Lagos
    - Top 100 de temas musicales de Spotify 2010 a 2019
 * GRUPO B:
    - FIFA 2021 Complete Player Dataset

## Lagos

##### Proceso Paso a Paso

Primero la importacion de los modulos os y csv que seran necesarios para operar sobre los archivos

Tambien se importa desde el archivo config.py la ruta FILES_ROOT = ```os.path.abspath(os.path.join(ROOT, "files"))```

In [1]:
import os
import csv
from config import FILES_ROOT

### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ```def pasaje_sexagesimal(linea):```

Esta función es utilizada para convertir la columna de coordenadas en grados decimales, para lo cual hay que realizar una conversión desde el sistema sexagesimal a grados decimales.

Los grados de la coordenada quedan como parte entera, los minutos se dividen por 60, y los segundos se dividen por 3600, luego se suman obteniendo asi las coordenadas expresadas en grados decimales.

Notar que todas las coordenadas estan expresadas en 2 posiciones (una para la latitud y otra para la longitud, por ende al string final de la coordenada se le agrega un "-" seguido de la letra inicial del punto cardinal al que hace referencia, es decir S O N E , siendo este el mismo que estaba en el dato original)

In [2]:
def pasaje_sexagesimal(linea):
    """ toma la coordenada por parametro, la "estandariza" reemplazando los simbolos de grados minutos y segundos
    por un caractger especial elegido.
    Luego hace un split sobre cada string utilizando dicho caracter especial y asi obteniendo en una lista los elementos 
    correspondientes a grados - minutos - segundos - punto cardinal en este orden, para finalmente realizar las conversiones
    casteando de int a str para las operaciones y volviendo a castear a str antes de retornar el valor completo procesado"""
    coordenadas = list(map(lambda coord: coord.replace('°', "-").replace("'","-").replace('"', "-"), linea.split(" ")))
    coordenadas = list(map(lambda elem : str(int(elem[0]) + int(elem[1])/60 + int(elem[2])/3600) + "-" + elem[3] , map(lambda elem: elem.split('-'), coordenadas)))
    return " ".join(coordenadas)

#### ```def filtrado_lagos(csv_reader):```

Recibe el csv Reader (sin el encabezado), toma los datos en una lista, y luego realiza los cambios para obtener los datos necesarios para el juego.

- Se Mappea ordenando las lineas de acuerdo a la salida que se precisa para su posterior utilizacion: "Ubicación”, “Superficie (km²)”, “Profundidad máxima (m)”, “Profundidad media (m)”, “Coordenadas” y “Nombre” .
- La columna de las coordenadas, se pasa como parametro a la funcion ```pasaje_sexagesimal(linea[5])``` para asi guardar luego del mappeo las coordenadas ya convertidas a grados decimales.
- Finalmente se retorna la estructura de ```datos``` que es una Lista de Listas (donde cada lista interna correponde a la informacion de un lago, con sus campos en strings)


In [3]:
def filtrado_lagos(csv_reader):
   """Recibe el csv Reader (sin el encabezado), toma los datos en una lista, luego :
   Primero Mappea sobre la lista de datos del csv reader en la columna 5 correspondiente a las coordenadas y le aplica la
   funcion de conversion de sistema sexagesimal a grados decimales a cada elemento, para luego finalmente devolver una 
   lista de strings uniendo latitud y longitud por cada elemento dado en las coordenadas de la variable datos.
   Finalmente reemplaza en cada una de las lineas de datos, la columna correspondiente a las coordenadas por el nuevo 
   formato de esta en grados decimales y retorna una lista de listas que contienen strings dentro."""

   return list(map(lambda linea: [linea[1], linea[2], linea[3], linea[4], pasaje_sexagesimal(linea[5]), linea[0]], list(csv_reader)))
  
   

#### ```def crear_lagos(csv_writer, lagos, encabezado):```
Recibe el csv_writer, la lista de lagos (lista de listas, cada lista interna es un lago y cada string dentro de esta lista es un dato correspondiente a dicho lago), y el encabezado original.

- Primero: realiza el corrimiento de los elementos del encabezado al nuevo formato de columnas que se necesita (anteriormente mencionado).
- Segundo: Escribe el nuevo archivo en el formato que se necesita para el juego, primero el encabezado y luego iterando sobre cada uno de los elementos de la lista de lagos, escribiendo linea a lineea en el nuevo csv

In [4]:
def crear_lagos(csv_writer, lagos, encabezado):
    """ Realiza los corrimientos en la lista de lagos y encabezado original, y luego crea un csv escribiendo
    linea por linea los datos en  el orden que se los necesitara posteriormente"""
    encabezado = [encabezado[1].title(), encabezado[2].title(), encabezado[3].title(), encabezado[4].title(), encabezado[5].title(), encabezado[0].title()]
    csv_writer.writerow(encabezado)
    for elem in lagos:
        csv_writer.writerow(elem)

## 2. Declaracion de las rutas a utilizar dentro de la estructura de archivos del juego

La estructura cuenta con una carpeta llamada "files".

En dicha carpeta, se encuentra una carpeta con el nombre "datasets_base" en la cual se almacenan los datasets originales.

Los datasets modificados y listos para la utilizacion de acuerdo a los requerimientos del juego, seran guardados dentro de la carpeta "files" en una carpeta llamada "datasets_final"

Es decir, desde donde se ejecuta el programa:

. >files>datasets_base                      -   aqui se ecuentran los archivos csv originales

. >files>datasets_final                     -   aqui se almacenaran los archivos csv procesados y posteriormente utilizados para la funcionalidad del juego

In [5]:
files = "files"
origen = "datasets_base"
destino = "datasets_final"

ruta_completa = FILES_ROOT


## 3. Apertura del archivo ORIGINAL y su procesamiento

Se abre el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Se Inicializa el csv reader y se lee la primera linea(el encabezado) que es guardado en una variable para su posterior uso.

Se guarda en una variable la lista de retorno al invocar a la funcion ```filtrado(csv_reader)``` enviando el csv_reader como parametro.

In [6]:

with open (os.path.join(ruta_completa,origen, "lagos.csv"), 'r', encoding='utf-8') as archivo:
    csv_reader = csv.reader(archivo, delimiter=",")
    encabezado = next(csv_reader)
    lagos_final = filtrado_lagos(csv_reader)

## 4. Creacion del archivo FINAL

Una vez realizado el procesamiento de datos sobre el archivo original, se procede a la escritura del archivo Final de destino, el cual pasara a ser el archivo utilizado posteriormente para implementar la jugabilidad del software.

Se inicializa un csv writer, y se invoca a la funcion ```crear_lagos(csv_writer, lagos_final, encabezado)``` pasando como parametro el csv writer, la lista de lagos procesada anteriormente y el encabezado del archivo original

In [7]:
with open(os.path.join(ruta_completa,destino, "lagos_filtrado.csv"), 'w', encoding='utf-8', newline = '') as archivo_filtrado:
    csv_writer = csv.writer(archivo_filtrado)
    crear_lagos(csv_writer, lagos_final, encabezado)

## Top 100 de temas musicales de Spotify 2010 a 2019

##### Proceso Paso a Paso

La importacion de los modulos os y csv que seran necesarios para operar sobre los archivos ya fue realizada anteriormente por lo que no es necesario hacerlo nuevamente, ya que se realiza una sola vez por sesion de interprete.


### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ``` def filtrado_spotify(csv_reader):```

Recibe por parametro el csv reader y crea una lista de listas con Strings adentro con los datos del mismo, quedando asi una lista para cada linea del csv dentro de la lista principal. 

Filtra y ordena la lista de quedandose en principio con las 6 columnas necesarias y desechando el resto de las columnas que no seran utilizadas posteriormente, al mismo tiempo que deja en la variable ```datos``` una lista ya ordenada por las columnas que luego se necesitaran escribir en el csv final para la jugabidildad de la aplicacion.

Realiza los cambios necesarios en la coluna de "top genre" pasando a mayuscula las expeciones explicitadas: " EDM, DFW, UK LGBTQ+ " y posteriormente a title case todos los generos musicales, contemplando y tratando las excepciones de manera diferencial para no alterar su estilo mediante la invocacion a la funcion ```estilo_generos(genero)```.

Notar que: las excepciones k-pop y r&b no son tratadas como excepciones o casos especiales dado que al pasarlas a title case como todas las otras, tomaran el formato deseado

Filtra aquellas lineas que tengan la primer columna vacia para evitar errores de insercion en el archvio csv final.

Retorna una lista de listas con los campos ya ordenados y listos para escribir en el archivo csv final

In [8]:
def filtrado_spotify(csv_reader):
    """Recibe por parametro el csv reader y crea una lista con los datos del mismo. Filtra y ordena la lista de quedandose
    en principio con las 6 columnas necesarias y desechando el resto de las columnas que no seran utilizadas posteriormente,
    dejando en la variable datos una lista ya ordenada por las columnas que luego se necesitaran escribir en el csv final
    Realiza los cambios necesarios en la coluna de "top genre" pasando a title case y contemplando las excepciones EDM, DFW, UK LGBTQ+
    k-pop y r&b no son tratadas como excepciones especiales dado que al pasarlas a title case como todas las otras, tomaran el formato
    deseado
    Filtra aquellas lineas que tengan la primer columna vacia"""
    siglas = ['edm', 'dfw', 'uk', 'lgbtq+']
    upper_case = list(map(lambda elem: elem.upper(), siglas))
    datos = list(filter(lambda linea: linea[0]!= "" ,map(lambda linea: [estilo_generos(linea[2].replace(siglas[0], upper_case[0]).replace(siglas[1], upper_case[1]).replace(siglas[2], upper_case[2]).replace(siglas[3], upper_case[3])),linea[16], linea[3],linea[15], linea[5], linea[1]], list(csv_reader))))
    
    return datos


#### ``` def estilo_generos(genero, upper=['EDM', 'DFW', 'UK', 'LGBTQ+']):```

Funcion auxiliar que evalua si el Genero pasado por parametro - String - contiene alguna de las excepciones de estilo para los nombres de los generos, que estan inicializadas por default.

Si el genero contiene alguna de las excepciones, entonces la funcion evalua si el genero tiene mas palabras que la que dispara la excepcion. De ser asi, el resto de las palabras que componen el genero son pasadas a title case, conservando las mayusculas de la excepcion y uniendo todo a string nuevamente antes de retornarlo en el formato que se desea.

Por el contrario, si no es ninguna de las excepciones por default, la funcion pasara a "title case" el string recibido y lo devolvera.

In [9]:
def estilo_generos(genero, upper_case=['EDM', 'DFW', 'UK', 'LGBTQ+']):
    """ Funcion auxiliar que evalua si el genero pasado por parametro - String -  contiene alguna de las excepciones
    de estilo para los nombres. Si contiene la excepcion, separa sus palabras en una lista e itera sobre estas pasando
    a title case aquellas palabras que no sean ninguna de las excepciones.
    Si no es un genero de excepcion, lo pasa a title case y lo retorna"""
    if (upper_case[0] in genero or upper_case[1] in genero or upper_case[2] in genero or upper_case[3] in genero):
        aux = genero.split(' ')             
        for i in range(len(aux)):
            if aux[i] not in upper_case:
                aux[i] = aux[i].title()
        return ' '.join(aux)
    else:
        return genero.title()

#### ```def crear_spotify(csv_writer, lista, encabezado):```
Recibe el csv_writer, la lista de top 100 temas musicales (lista de listas, cada lista interna es un tema musical y cada string dentro de esta lista es un dato correspondiente a dicho tema musical), y el encabezado original.

- Primero: realiza el corrimiento de los elementos del encabezado al nuevo formato de columnas que se necesita (anteriormente mencionado).

- Segundo: Escribe el nuevo archivo en el formato que se necesita para el juego, primero el encabezado y luego iterando sobre cada uno de los elementos de la lista de temas musicales, escribiendo linea a lineea en el nuevo csv

In [10]:
def crear_spotify(csv_writer, spotify_list, encabezado):
    """ Realiza los corrimientos en el encabezado original, y luego crea un csv escribiendo
    linea por linea los datos en  el orden que se los necesitara posteriormente"""
    encabezado = [encabezado[2].title(),encabezado[16].title(), encabezado[3].title(),encabezado[15].title(), encabezado[5].upper(), encabezado[1].title()]
    csv_writer.writerow(encabezado)
    for elem in spotify_list:
        csv_writer.writerow(elem)

## 3. Apertura del archivo ORIGINAL y su procesamiento

Se abre el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Se Inicializa el csv reader y se lee la primera linea(el encabezado) que es guardado en una variable para su posterior uso.

Se guarda en una variable la lista de retorno al invocar a la funcion ```filtrado(csv_reader)``` enviando el csv_reader como parametro.

In [11]:
with open (os.path.join(ruta_completa,origen, "spotify.csv"), 'r', encoding='utf-8') as archivo:
    csv_reader = csv.reader(archivo, delimiter=",")
    encabezado = next(csv_reader)
    spotify_list = filtrado_spotify(csv_reader)


## 4. Creacion del archivo FINAL

Una vez realizado el procesamiento de datos sobre el archivo original, se procede a la escritura del archivo Final de destino, el cual pasara a ser el archivo utilizado posteriormente para implementar la jugabilidad del software.

Se inicializa un csv writer, y se invoca a la funcion ```crear_spotify(csv_writer, spotify_list, encabezado)``` pasando como parametro el csv writer, la lista de top 100 canciones spotify procesada anteriormente y el encabezado del archivo original

In [12]:
with open(os.path.join(ruta_completa,destino, "spotify_filtrado.csv"), 'w', encoding='utf-8', newline = '') as archivo_filtrado:
    csv_writer = csv.writer(archivo_filtrado)
    crear_spotify(csv_writer, spotify_list, encabezado)

## FIFA 2021 Complete Player Dataset

##### Proceso Paso a Paso

La importacion de los modulos os y csv que seran necesarios para operar sobre los archivos ya fue realizada anteriormente por lo que no es necesario hacerlo nuevamente, ya que se realiza una sola vez por sesion de interprete.


### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ``` def potencial(score):```

Recibe por parametro el String score correspondiente al valor de la columna del csv.

Lo convierte a integer para luego hacer las comparaciones y asignarle de acuerdo al valor en la escala, un adjetivo calificativo que luego reemplazara al valor numerico en la columna de potencial del csv final.

In [14]:
def potencial(score):
    """ Recibe por parametro el String score correspondiente al valor de la columna del csv. Lo convierte a integer para luego hacer las comparaciones y asignarle de acuerdo
    al valor en la escala, un adjetivo calificativo que luego reemplazara al valor numerico en la columna de potencial del csv final. Se retorna dicho valor expresado como 
    adjetivo calificativo. """

    score = int(score)
    if  score < 60: 
        score = "Regular" 
    elif score <= 79:
        score = "Bueno"
    elif score <= 89:
        score = "Muy Bueno"
    else:
        score = "Sobresaliente"
    return score


#### ``` def posicion(posicion):```

Recibe por parametro el String con la posicion o posiciones del jugador de futbol.

1. Lo divide mediante el caracter "|" que separa las multiples posiciones del jugador, obteniendo asi una lista de posiciones
2. Para cada posible posicion del jugador, segun sean las siglas de la misma, se reemplaza su valor por la descripcion en español de la misma
3. Vuelve a unir mediante el caracter especial "|" las posiciones ya traducidas y retorna el dicho String


In [15]:

def posicion(posicion):
    """"  Recibe por parametro el String con la posicion o posiciones del jugador de futbol.
    1. Lo divide mediante el caracter "|" que separa las multiples posiciones del jugador, obteniendo asi una lista de posiciones
    2. Para cada posible posicion del jugador, segun sean las siglas de la misma, se reemplaza su valor por la descripcion en español de la misma
    3. Vuelve a unir mediante el caracter especial "|" las posiciones ya traducidas y retorna el dicho String"""
    
    posicion = posicion.split('|')
    for pos in range(0, len(posicion)):
        match posicion[pos]:
            case "GK": posicion[pos] = "Arquero"
            case "ST": posicion[pos] = "Segundo Delantero"
            case "CF": posicion[pos] = "Delantero Centro"
            case "RW": posicion[pos] = "Extremo Derecho"
            case "LW": posicion[pos] = "Extremo Izquierdo"
            case "LS": posicion[pos] = "Delantero Izquierdo"
            case "RS": posicion[pos] = "Delantero Derecho"
            case "CAM": posicion[pos] = "Centrocampista Ofensivo Central"
            case "RAM": posicion[pos] = "Centrocampista Ofensivo Derecho"
            case "LAM": posicion[pos] = "Centrocampista Ofensivo Izquierdo"
            case "LM": posicion[pos] = "Mediocampista Izquierdo"
            case "LCM": posicion[pos] = "Mediocampista Central Izquierdo"
            case "CM": posicion[pos] = "Mediocampista Central"
            case "RCM": posicion[pos] = "Mediocampista Central Derecho"
            case "RM": posicion[pos] = "Mediocampista Derecho"
            case "LDM": posicion[pos] = "Mediocampista Defensivo Izquierdo"
            case "CDM": posicion[pos] = "Mediocampista Defensivo Central"
            case "RDM": posicion[pos] = "Mediocampista Defensivo Derecho"
            case "RWB": posicion[pos] = "Defensor Extremo Derecho"
            case "RB": posicion[pos] = "Defensor Lateral Derecho"
            case "RCB": posicion[pos] = "Defensor Central Derecho"
            case "CB": posicion[pos] = "Defensor Central"
            case "LCB": posicion[pos] = "Defensor Central Izquierdo"
            case "LB": posicion[pos] = "Defensor Lateral Izquierdo"
            case "LWB": posicion[pos] = "Defensor Extremo Izquierdo"
            case "SW": posicion[pos] = "Libero"
    return " | ".join(posicion)


#### ``` def filtrado_fifa(csv_reader):```

Recibe por parametro el csv reader y crea una lista de listas con Strings adentro con los datos del mismo, quedando asi una lista para cada linea del csv dentro de la lista principal. 

Mappea el csv reader al mismo tiempo que ordena los campos de acuerdo a el orden que se necesita para el csv final : “Team”, “Nationality”, “Position”, “Age”, “Potential” y “Name”.

Para la columna posicion, se invoca a la funcion ```posicion(linea[3])``` que modifica y traduce al español dicho dato, y para la columna del potencial tambien se invoca a la funcion que hace el procesamiento de dicha informacion ```potencial(linea[7])```.

Se retorna una Lista de listas, donde cada una de estas ultimas correspondera a un jugador y sus datos en formato Lista de Strings ordenados como anteriormente se explicito.

In [20]:
def filtrado_fifa(csv_reader):
    """ Recibe por parametro el csv reader y crea una lista de listas con Strings adentro con los datos del mismo, quedando asi una lista para cada linea del csv dentro de la lista principal. 
    Mappea el csv reader al mismo tiempo que ordena los campos de acuerdo a el orden que se necesita para el csv final : “Team”, “Nationality”, “Position”, “Age”, “Potential” y “Name”.
    Para la columna posicion, se invoca a la funcion posicion(pos) que modifica y traduce al español dicho dato, y para la columna del potencial tambien se invoca a la funcion que
    hace el procesamiento de dicha informacion potencial(numero).
    Se retorna una Lista de listas, donde cada una de estas ultimas correspondera a un jugador y sus datos en formato Lista de Strings ordenados como anteriormente se explicito."""
    return list(map(lambda linea: [linea[8],linea[2], posicion(linea[3]), linea[5], potencial(linea[7]), linea[1]], list(csv_reader)))


#### ```def crear_fifa(csv_writer, lista, encabezado):```
Recibe el csv_writer, la lista de jugadores de Fifa (lista de listas, cada lista interna es un jugador y cada string dentro de esta lista es un dato correspondiente a dicho jugador), y el encabezado original.

- Primero: realiza el corrimiento de los elementos del encabezado al nuevo formato de columnas que se necesita (anteriormente mencionado).

- Segundo: Escribe el nuevo archivo en el formato que se necesita para el juego, primero el encabezado y luego iterando sobre cada uno de los elementos de la lista de temas musicales, escribiendo linea a lineea en el nuevo csv

In [17]:
def crear_spotify(csv_writer, fifa_set, encabezado):
    """ Realiza los corrimientos en el encabezado original, y luego crea un csv escribiendo
    linea por linea los datos en  el orden que se los necesitara posteriormente"""
    encabezado = [encabezado[8].title(),encabezado[2].title(), encabezado[3].title(), encabezado[5].title(), encabezado[7].title(), encabezado[1].title()]
    csv_writer.writerow(encabezado)
    for elem in fifa_set:
        csv_writer.writerow(elem)

## 3. Apertura del archivo ORIGINAL y su procesamiento

Se abre el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Se Inicializa el csv reader y se lee la primera linea(el encabezado) que es guardado en una variable para su posterior uso.

Se guarda en una variable la lista de retorno al invocar a la funcion ```filtrado(csv_reader)``` enviando el csv_reader como parametro.

In [18]:
with open (os.path.join(ruta_completa,origen, "FIFA-21.csv"), 'r', encoding='utf-8') as archivo:
    csv_reader = csv.reader(archivo, delimiter=";")
    encabezado = next(csv_reader)
    fifa_set = filtrado_fifa(csv_reader)

## 4. Creacion del archivo FINAL

Una vez realizado el procesamiento de datos sobre el archivo original, se procede a la escritura del archivo Final de destino, el cual pasara a ser el archivo utilizado posteriormente para implementar la jugabilidad del software.

Se inicializa un csv writer, y se invoca a la funcion ```crear_fifa(csv_writer, fifa_set, encabezado)``` pasando como parametro el csv writer, la lista de jugadores Fifa procesada anteriormente y el encabezado del archivo original

In [19]:
with open(os.path.join(ruta_completa,destino, "fifa_filtrado.csv"), 'w', encoding='utf-8', newline = '') as archivo_filtrado:
    csv_writer = csv.writer(archivo_filtrado)
    crear_spotify(csv_writer, fifa_set, encabezado)